In [1]:
%%loadFromPOM
<repositories>
    <repository>
        <id>jitpack.io</id>
        <url>https://jitpack.io</url>
    </repository>
    <repository>
        <id>sovrin</id>
        <url>https://repo.sovrin.org/repository/maven-public</url>
    </repository>
</repositories>

<dependency>
    <groupId>com.github.Sirius-social</groupId>
    <artifactId>sirius-sdk-java</artifactId>
    <version>ignore_tests-SNAPSHOT</version>
</dependency>

In [2]:
import com.sirius.sdk.agent.aries_rfc.feature_0095_basic_message.Message;
import com.sirius.sdk.agent.listener.Event;
import com.sirius.sdk.agent.listener.Listener;
import com.sirius.sdk.agent.n_wise.NWiseParticipant;
import com.sirius.sdk.agent.n_wise.messages.Invitation;
import com.sirius.sdk.agent.n_wise.messages.LedgerUpdateNotify;
import com.sirius.sdk.hub.Context;
import io.reactivex.rxjava3.core.Observable;
import io.reactivex.rxjava3.functions.Consumer;
import io.reactivex.rxjava3.subjects.ReplaySubject;
import com.sirius.sdk.hub.MobileContext;
import com.sirius.sdk.hub.MobileHub;
import com.sirius.sdk.hub.CloudContext;
import com.sirius.sdk.hub.CloudHub;

import java.util.ArrayList;
import java.util.List;

public class NWiseClient {
    
    public NWiseClient(CloudHub.Config config, String nickname) {
        this.nickname = nickname;
        context = new CloudContext(config);
    }

    public static class NWiseMessage {
        public Message message;
        public String nWiseInternalId;
        public String senderDid;
    }

    Context context = null;
    boolean loop = false;
    String nickname;
    List<NWiseMessage> receivedMessages = new ArrayList<>();
    ReplaySubject<Event> observable = ReplaySubject.create();

    public void start() {
        if (!loop) {
            loop = true;
            new Thread(() -> routine()).start();
        }
    }

    public void stop() {
        if (context != null) {
            loop = false;
            context.close();
        }
    }

    public String createNWise(String nWiseName) {
        return context.getNWiseManager().create(nWiseName, nickname);
    }

    public String acceptInvitation(Invitation invitation) {
        return context.getNWiseManager().acceptInvitation(invitation, nickname);
    }

    public Invitation createNWiseInvitation(String internalId) {
        return context.getNWiseManager().createInvitation(internalId);
    }

    public boolean updateNWise(String internalId) {
        return context.getNWiseManager().update(internalId);
    }

    public List<NWiseParticipant> getNWiseParticipants(String internalId) {
        return context.getNWiseManager().getParticipants(internalId);
    }

    public boolean sendNWiseMessage(String internalId, Message message) {
        return context.getNWiseManager().send(internalId, message);
    }

    public List<NWiseMessage> getReceivedMessages() {
        return receivedMessages;
    }

    public Observable<Event> getEvents() {
        return observable;
    }

    public NWiseParticipant getMe(String internalId) {
        return context.getNWiseManager().getMe(internalId, context);
    }

    public boolean leave(String internalId) {
        return context.getNWiseManager().leave(internalId, context);
    }

    protected void routine() {
        Listener listener = context.subscribe();
        listener.listen().blockingSubscribe(new Consumer<Event>() {
            @Override
            public void accept(Event event) {
                String nWiseId = context.getNWiseManager().resolveNWiseId(event.getSenderVerkey());
                NWiseParticipant sender = context.getNWiseManager().resolveParticipant(event.getSenderVerkey());
                String senderNickname = sender != null ? sender.nickname : "Unknown";
                System.out.println(nickname + " received new message from " + senderNickname + " : " + event.message().getMessageObj());
                if (event.message() instanceof Message) {
                    if (nWiseId != null && sender != null) {
                        Message message = (Message) event.message();
                        NWiseMessage nWiseMessage = new NWiseMessage();
                        nWiseMessage.message = message;
                        nWiseMessage.nWiseInternalId = context.getNWiseManager().resolveNWiseId(event.getSenderVerkey());
                        nWiseMessage.senderDid = sender.did;
                        receivedMessages.add(nWiseMessage);
                    }
                } else if (event.message() instanceof LedgerUpdateNotify) {
                    context.getNWiseManager().getNotify(event.getSenderVerkey());
                }
                observable.onNext(event);
            }
        }, new Consumer<Throwable>() {
            @Override
            public void accept(Throwable throwable) throws Throwable {
                throwable.printStackTrace();
            }
        });
    }
}


In [3]:
import org.json.JSONObject;
import com.sirius.sdk.hub.MobileHub;
import com.sirius.sdk.hub.CloudHub;
import com.sirius.sdk.agent.aries_rfc.feature_0160_connection_protocol.messages.Invitation;
import com.sirius.sdk.encryption.P2PConnection;
import java.nio.charset.StandardCharsets;

JSONObject walletConfig = new JSONObject().
    put("id", UUID.randomUUID()).
    put("storage_type", "default");
JSONObject walletCredentials = new JSONObject().
    put("key", "8dvfYSt5d1taSd6yJdpjq4emkwsPDDLYxkNFysFD2cZY").
    put("key_derivation_method", "RAW");

CloudHub.Config aliceConfig = new CloudHub.Config();
aliceConfig.serverUri = "https://agents.socialsirius.com";
aliceConfig.credentials = "s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rwsta0b6XT84KQyxKcKkysg==".getBytes(StandardCharsets.UTF_8);
aliceConfig.p2p = new P2PConnection(
    "HYjGQGEtCbLisC2W3eeYxi196ojLV8sSdmGAAHt1Riry",
    "4MFTdbvvUVnNkQpU7UHJXz1Qea9WqvDXgbkxNetoBhCob45VUDS3Ef5fN3nLBqkGNSG5RftK9RidY7HxYcBp38Lm",
    "9b5fhAxyXoWP3ZHtpvKEFUf8o47uNcysuSd1c1VDq2yo");

CloudHub.Config bobConfig = new CloudHub.Config();
bobConfig.serverUri = "https://agents.socialsirius.com";
bobConfig.credentials = "s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+roqlm6OClcBVahrlBC6d7pg==".getBytes(StandardCharsets.UTF_8);
bobConfig.p2p = new P2PConnection(
    "BdVo9Ha72yyQmrfqLRn3bAfxj8wgf2uy4Rz7hYvzaf9e",
    "3spH9x8K4ojQ4qKmqXFL67hK96DvMNAR8oSJhAx3QS2etHPd3xGcsdKZoinu8fpAueHMmbiB9ZLzwyJpXPrDdyj2",
    "Hju2j1nh5EHk8VHQKHyoHWZrEJbimkVTdYZbhnfMLrmg");

CloudHub.Config carolConfig = new CloudHub.Config();
carolConfig.serverUri = "https://agents.socialsirius.com";
carolConfig.credentials = "s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rdefbzDIW5SbyYbkI+UYP/Q==".getBytes(StandardCharsets.UTF_8);
carolConfig.p2p = new P2PConnection(
    "DQTDLvkSAjGYmKG5a9EvgvUwJzKXQfQuqxwgYHz6H9jE",
    "2dUPJxWVbNU1VhsUFAkVNLeq3kG7TghBLum8atfy6C3QAkG4GAYxAWEU4NMy3Qi9BGVdehP7UATaDdhw8g79A2jJ",
    "HCNPicRwYQHhdQ7Z5nNQ8kaAQkLeetmLctfyZ5GVxDy9");

NWiseClient alice = new NWiseClient(aliceConfig, "Alice");
NWiseClient bob = new NWiseClient(bobConfig, "Bob");
NWiseClient carol = new NWiseClient(carolConfig, "Carol");

In [4]:
alice.start();
bob.start();
carol.start();

In [5]:
String aliceNWiseInternalId = alice.createNWise("new n-wise");

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.


In [6]:
aliceNWiseInternalId

4097e16e-65e1-43f0-8a6f-a7d2ccce6d1d

In [7]:
import com.sirius.sdk.agent.n_wise.messages.Invitation;
Invitation invitationAliceToBob = alice.createNWiseInvitation(aliceNWiseInternalId);

In [8]:
invitationAliceToBob.getMessageObj()

{"invitationPrivateKeyBase58":"4Fh3HRuoUSU3DbTHkuE6XiBpNSD3322YvHHHL7xm1ko2tJZ6SEeQSa8tyyYE8i11GgCjWnkScXNFk7J6w1myR8hx","@type":"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/n-wise/1.0/invitation","invitationKeyId":"6oQJrKRCTcNiPiXJxw9KgbjBDVr6XYPCpxQe6FKSrF7x","ledgerType":"iota@v1.0","@id":"6a1780d1-2f89-4708-92ae-b7bf438fb36f","label":"new n-wise","attach":{"tag":"AvMwEiKeFoXkss1peZh1f1cxcx4vpKWusaLxGURuPUKC"}}

In [9]:
String bobNWiseInternalId = bob.acceptInvitation(invitationAliceToBob);

In [10]:
alice.updateNWise(aliceNWiseInternalId);
alice.getNWiseParticipants(aliceNWiseInternalId).size()

Alice received new message from Bob : {"@type":"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/n-wise/1.0/ledger-update-notify","@id":"3b1f33d4-a97f-4a41-bfbd-28dddb031c67"}


2

In [12]:
Message bobToAlice = Message.builder().setContent("hello world!").build();
bob.sendNWiseMessage(bobNWiseInternalId, bobToAlice);

true

In [13]:
alice.updateNWise(aliceNWiseInternalId);
alice.getNWiseParticipants(aliceNWiseInternalId).size()

2